In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn import *
from sklearn.preprocessing import Imputer
from sklearn.svm import SVC

In [2]:
# Loading the files
train_features = pd.read_csv("train_features.csv") # training X
train_labels = pd.read_csv("train_labels.csv") # training Y
test_features = pd.read_csv("test_features.csv") # testing X

In [3]:
# Getting the data we need for subtask 1
train_Y = np.array(train_labels.loc[: , "pid":"LABEL_EtCO2"])

In [4]:
def preprocess(data):
    # Transforming train features into an array where each element is one patient
    featureaary = np.array(data)
    number_of_hours = 12
    patients = np.vsplit(featureaary , int(data.shape[0]/number_of_hours))
    patients_array = np.array(patients)
    
    # We are creating a new matrix (Phi) that contains statistical features for every patient: pid, age, mean, var, max, min,median
    features = np.zeros([int(data.shape[0]/number_of_hours), 2+5*34]) # no of patients, no of features* no of columns names(age, calcium, bolirubin...)
    # columns are: pid, age, mean(of all featuers), var, max, min, median
    
    features[:,0] = patients_array[:,0,0] #Patient id
    features[:,1] = patients_array[:,0,2] # Age

    means = np.nanmean(patients_array[:, :, 3:], axis=1)
    statistical_features = int(data.shape[1])-3  # Number of columns - pid - age - time
    
    features[:,2:statistical_features+2] = means
    
    variances = np.nanvar(patients_array[:, :, 3:], axis=1);
    features[:,statistical_features+2:2*statistical_features+2] = variances
    
    maxs = np.nanmax(patients_array[:, :, 3:], axis=1);
    features[:,2*statistical_features+2:3*statistical_features+2] = maxs
    
    mins = np.nanmin(patients_array[:, :, 3:], axis=1);
    features[:,3*statistical_features+2:4*statistical_features+2] = mins
    
    medians = np.nanmedian(patients_array[:, :, 3:], axis=1);
    features[:,4*statistical_features+2:5*statistical_features+2] = medians
    
    #Replacing the NaN values with the mean of all patients
    # Calculate the mean of each features of all patients
    all_means = np.nanmean(features, axis=0)
    
    # find indexes where NaNs are and replace them by the column mean
    indexes = np.where(np.isnan(features))
    features[indexes] = np.take(all_means, indexes[1])
        
    return features

In [ ]:
train_X = preprocess(train_features)

C:\Users\paulm\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app
C:\Users\paulm\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: Degrees of freedom <= 0 for slice.
C:\Users\paulm\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: All-NaN slice encountered
C:\Users\paulm\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: All-NaN slice encountered
C:\Users\paulm\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3250: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


We can ignore these warnings as the result still make sense. We get NaN for columns that contist only of NaN's.

now, we have a nice $\Phi$ matrix, we need to take care of the remaining NaN's

In [ ]:
test_X = preprocess(test_features)

C:\Users\paulm\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app
C:\Users\paulm\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: Degrees of freedom <= 0 for slice.
C:\Users\paulm\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: All-NaN slice encountered
C:\Users\paulm\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: All-NaN slice encountered


Looks good, we finally have usable features to train the model!

In [ ]:
svm = SVC(kernel='linear', probability=True)
svm.fit(train_X, train_Y[:,1])
output_proba = svm.predict_proba(test_X)

In [ ]:
np.shape(train_Y)

In [ ]:
np.shape(train_X)

In [ ]:
output_proba